In [1]:
# This pipeline will test collaborative filtering as a means of guessing
# which users will interact in the future.
# Definition of steps:
# Import data,
# create the collaborative filtering scheme, and pass the data through the collaborative filter.
# use the scheme to predict which pairs will interact at a future time.

In [1]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('TKAgg')
from matplotlib import pyplot as plt
from scipy.spatial.distance import cosine

from sklearn.decomposition import PCA
from scipy.stats import pearsonr
from scipy.sparse import dok_matrix, csr_matrix
import scipy
from sklearn.cluster import KMeans
from collections import Counter
from sklearn.metrics import roc_curve, auc

In [7]:
dftrain = pd.read_csv('data/txTripletsCounts.txt',
                      header=None,
                      index_col=None,
                      sep=' ',
                      names=['sender','receiver','transaction'])
dftest = pd.read_csv('data/testTriplets.txt', 
                     header=None, 
                     index_col=None, 
                     sep=" ", 
                     names=['sender', 'receiver', 'transaction'])
dftrain['transaction'].describe()

count    3.348026e+06
mean     4.725741e+00
std      1.284948e+02
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      4.163900e+04
Name: transaction, dtype: float64

In [4]:
dftrain # lets see what our data looks like, we can see that its sender, receiver, transaction counts

,sender,receiver,transaction
0,0,1,3
1,0,13,1
2,0,37,1
3,0,51,1
4,0,438481,1
5,1,0,3
6,1,4,354
7,1,10,2602
8,1,11,2689
9,1,12,1


In [5]:
# Our collab filtering pipeline will go as follows (to test)
# first we will construct the count matrix of transmitter2receiver with counts
# then we will apply pca to get the lowest dimensionality representation
# then we will use collaborative filtering to predict if any receiver transmitter pair would have an interaction
# in the future.

In [3]:
# max sender numbe n = 444074
n = 444075
t2r_matrix = csr_matrix((dftrain['transaction'], (dftrain['sender'], dftrain['receiver'])), shape=(n,n), dtype=float)
t2r_matrix[364,:].toarray()

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.]])

In [4]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
lda = LDA(n_topics=10)

A = lda.fit_transform(t2r_matrix)
B = lda.components_()

KeyboardInterrupt: 

In [ ]:
pred = [np.sum(A[row['sender'],:] * B[:,row['receiver']]) 
        for index,row in dftest.iterrows()]
label = dftest['transaction']
fpr, tpr, thresholds = roc_curve(label, pred)
roc_auc = auc(fpr, tpr)
print "Area under the ROC curve : %f" % roc_auc
matplotlib.rcParams['figure.figsize'] = (10, 10)
plt.plot(fpr, tpr, color='magenta', label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [4]:
# create adjacency-list graph G = (V, E)
N = 444075
G = [[] for i in range(N)]
for (index, sender, receiver, count) in dftrain.itertuples():
    G[sender].append(receiver)


In [5]:
# create transpose adjacency-list graph Gt = (V, E')
Gt = [[] for i in range(N)]
for (index, sender, receiver, count) in dftrain.itertuples():
    Gt[receiver].append(sender)

In [6]:
G_uni = [[] for i in range(N)]
for (index, sender, receiver, count) in dftrain.itertuples():
    G_uni[sender].append(receiver)
    G_uni[receiver].append(sender)

In [8]:
# Lets reduce dimensionality first!
from scipy.sparse.linalg import svds
epsilon = 1e-10
new_dim = 10
u, s, vt = svds(t2r_matrix, k=new_dim, tol=epsilon, which='LM')

In [9]:
v = vt.transpose()

In [10]:
d = 2
def isConnected(S, R, D):
    # S and R are points on the Graph G, and can be indexed via the adjacency-list
    def isConnectedRec(s_un, depthRemaining, isTranspose):
        # iterate through s_un, look at all their neighbors in the corresonding graph
        rec_set = set([])
        if depthRemaining == 0:
            return False
        for v in s_un:
            if isTranspose:
                for sender in Gt[v]:
                    rec_set.add(sender)
            else:
                for child in G[v]:
                    if child == R and depthRemaining % 2 == 1:
                        return True
                    rec_set.add(child)
        return isConnectedRec(rec_set, depthRemaining - 1, ~isTranspose)
    return isConnectedRec(set([S]), D, False)
isConnected(0, 2, 3)

True

In [11]:
from scipy.spatial.distance import cosine

def simConnected(S, R, D):
    threshold = 1.0
    R_vec = v[R]
    S_vec = u[S]
    # S and R are points on the Graph G, and can be indexed via the adjacency-list
    def simConnectedRec(s_un, depthRemaining, isTranspose):
        # iterate through s_un, look at all their neighbors in the corresonding graph
        rec_set = set([])
        if depthRemaining == 0:
            return False
        for vertex in s_un:
            if isTranspose:
                for sender in Gt[vertex]:
                    sender_vec = u[sender]
                    similarity_score = cosine(S_vec, sender_vec)
                    if similarity_score > threshold:
                        rec_set.add(sender)
            else:
                for child in G[vertex]:
                    if child == R and depthRemaining % 2 == 1:
                        return True
                    child_vec = v[child]
                    similarity_score = cosine(R_vec, child_vec)
                    if similarity_score > threshold:
                        rec_set.add(child)
        return simConnectedRec(rec_set, depthRemaining - 1, ~isTranspose)
    return simConnectedRec(set([S]), D, False)
simConnected(1,16,3)

True

In [12]:
import math
mean_score = 100343.0
def volatilityScore(v, isSender):
    # given any vertex v, assess its volatility
    # assumption is that high volatility on a sender or receiver side has implications about their ability to meet
    # or form new connections
    graph = G
    graph_flip = Gt
    if not isSender:
        graph = Gt
        graph_flip = G
        
    v_neigh_list = graph[v]
    score = len(v_neigh_list)
    for neigh in v_neigh_list:
        # you are going to get the indegree
        neigh_list = graph_flip[neigh]
        score = score + len(neigh_list) - 1 # otherwise you count yourself.
    return (score - mean_score)/mean_score
volatilityScore(4000, True)

# calculate mean volatility
# total_score = 0.0
# items = 0.0
# for v in range(len(G)):
#     items = items + 1
#     total_score = total_score + volatilityScore(v, True)

# mean_score = total_score / len(G)


0.9658072810260805

In [ ]:
dftest

In [16]:
# # Time to test this crap.
# pred = [isConnected(sender,receiver,3) 
#         for index,sender,receiver,transaction  in dftest.itertuples()]
pred = []
count = 0
for index,sender,receiver,transaction in dftest.itertuples():
    a = isConnected(sender,receiver,3)
    print count
    count = count + 1
    pred.append(a)

label = dftest['transaction']
fpr, tpr, thresholds = roc_curve(label, pred)
roc_auc = auc(fpr, tpr)
print "Area under the ROC curve : %f" % roc_auc
# matplotlib.rcParams['figure.figsize'] = (10, 10)
# plt.plot(fpr, tpr, color='magenta', label='ROC curve (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], 'k--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic example')
# plt.legend(loc="lower right")
# plt.show()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
Area under the ROC curve : 0.504934


In [17]:
# Test baseline volatility score metric
# # Time to test this crap.
# pred = [isConnected(sender,receiver,3) 
#         for index,sender,receiver,transaction  in dftest.itertuples()]
pred = []
count = 0
for index,sender,receiver,transaction in dftest.itertuples():
    a = volatilityScore(sender, True)
    b = volatilityScore(receiver, False)
    if a > -0.25 and b > -0.25:
        pred.append(1)
    else:
        pred.append(0)
#     print count
    count = count + 1

label = dftest['transaction']
fpr, tpr, thresholds = roc_curve(label, pred)
roc_auc = auc(fpr, tpr)
print "Area under the ROC curve : %f" % roc_auc
# matplotlib.rcParams['figure.figsize'] = (10, 10)
# plt.plot(fpr, tpr, color='magenta', label='ROC curve (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], 'k--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic example')
# plt.legend(loc="lower right")
# plt.show()

Area under the ROC curve : 0.627467


In [18]:
# first time trial, we will run pure collaborative content based filtering, and see what happens just to get a
# baseline measure done.
# # this will iterate row wise, so first we will compare the interactions of row 1 to row 2 and so forth.
# pearsonr_corr_matrix = dok_matrix((n,n))
# for i in range(n):
#     for j in range(n):
#         row_dense = np.squeeze((t2r_matrix.tocsr()[i,:]).toarray().flatten().transpose())
#         print row_dense.shape
#         row2_dense = np.squeeze((t2r_matrix.tocsr()[j,:]).toarray().flatten().transpose())
#         print row2_dense.shape
#         pearsonr_corr_matrix[i, j] = pearsonr(row_dense, row2_dense)[0] # grab just the correlation value
#         print i, j
# pearsonr_corr_matrix

In [19]:
# Lets reduce dimensionality first!
from scipy.sparse.linalg import svds
epsilon = 1e-10
new_dim = 100
u, s, vt = svds(t2r_matrix, k=new_dim, tol=epsilon, which='LM')
v = vt.transpose()

KeyboardInterrupt: 

In [13]:
# Third pipeline
# Given some R, get the senders it interacts with off the Gt graph
# Get the Rset that are in the interaction set for those senders
# compute the pearson similarity on those set of Receivers.
# take top 10 receivers, and see if S has interacted with them in a maximal fashion
def connectionByGraphSimilarity(S, R):
    R_vec = v[R]
    def getReceiverSet():
        sender_set = Gt[R]
        receiver_set = set()
        for sender in sender_set:
            for r in G[sender]:
                receiver_set.add(r)
        return receiver_set
    
    def computeReceiverSimilarity(receiver_set):
        # (score, index) to store the similarity list
        similarity_list = []
        for r in receiver_set:
            vec = v[r] # get the vector from the SVD matrix
            score = cosine(R_vec, vec)
            similarity_list.append((score, r))
        return sorted(similarity_list, key=lambda tup: tup[0])

    def isConnectedViaSimilar(scores, k, p):
        # take top k vectors, and check if the sender is in the set
        top_k_tuples = scores[-k:]
        connected_count = 0.0
        for (sc, index) in top_k_tuples:
            for sender in Gt[index]:
                if sender == S:
                    connected_count = connected_count + 1.0
#         print connected_count
#         print connected_count / float(k)
        if (connected_count / float(k)) > p:
            return True
        return False
        
    if S == R:
        return True
    
    r_set = getReceiverSet()
    sim_scores = computeReceiverSimilarity(r_set)
#     print sim_scores
    return isConnectedViaSimilar(sim_scores, 10, 0.25)
connectionByGraphSimilarity(8,32443)

False

In [14]:
def numeric_connectedBySimilarity(S,R):
    if connectionByGraphSimilarity(S, R):
        return 1
    else:
        return 0

In [ ]:
pred = []
count = 0
for index,sender,receiver,transaction in dftest.itertuples():
    if connectionByGraphSimilarity(sender, receiver):
        pred.append(1)
    else:
        pred.append(0)
#     print count
    count = count + 1

label = dftest['transaction']
fpr, tpr, thresholds = roc_curve(label, pred)
roc_auc = auc(fpr, tpr)
print "Area under the ROC curve : %f" % roc_auc

In [15]:
# Jaccard similarity measure
def jaccardSimilarity(S, R):
    S_neighbors = set(G_uni[S])
    R_neighbors = set(G_uni[R])
    SR_intersection = S_neighbors.intersection(R_neighbors)
    SR_union = S_neighbors.union(R_neighbors)
    return float(len(SR_intersection)) / float(len(SR_union))

In [ ]:
total_jaccard = 0.0
for i in range(0, 50):
    for j in range(0, 50):
        total_jaccard = total_jaccard + jaccardSimilarity(i, j)
print total_jaccard / 2500.0

In [ ]:
# find the median
jaccard_list = []
for i in range(0,50):
    for j in range(0,50):
        jaccard_list.append(jaccardSimilarity(i, j))

median_jaccard = np.median(jaccard_list)

In [ ]:
0.08 * median_jaccard

In [ ]:
pred = []
count = 0
for index,sender,receiver,transaction in dftest.itertuples():
    if jaccardSimilarity(sender, receiver) > 0:
        pred.append(1)
    else:
        pred.append(0)
#     print count
    count = count + 1

label = dftest['transaction']
fpr, tpr, thresholds = roc_curve(label, pred)
roc_auc = auc(fpr, tpr)
print "Area under the ROC curve : %f" % roc_auc

In [16]:
import math
def preferentialScore(S, R):
    S_neighbors = set(G_uni[S])
    R_neighbors = set(G_uni[R])
    return math.log(len(S_neighbors) * len(R_neighbors))

In [17]:
def jaccardAndPreferentialScore(S, R):
    return 0.9 * jaccardSimilarity(S, R) + 0.1 * preferentialScore(S, R)


In [18]:
def jaccardAndVolatilityScore(S, R):
    return 0.5 * jaccardSimilarity(S, R) + 0.5 * volatilityScore(S, R)

In [19]:
def jaccardAndVolatilityAndConnectionViaGraphSimilarity(S,R):
    if connectionByGraphSimilarity(S,R):
        return 0.33 + jaccardAndVolatilityScore(S,R)
    else:
        return jaccardAndVolatilityScore(S,R)

In [ ]:
1.39776250554 # median pref + jaccard

In [ ]:
0.127677740241 # median volatility + jaccard

In [ ]:
median_list = []
for i in range(0, 50):
    for j in range(0, 50):
        median_list.append(jaccardAndVolatilityScore(i, j))
median_prefAndJaccard = np.median(median_list)
print median_prefAndJaccard

In [ ]:
median_list = []
for i in range(40, 50):
    for j in range(30, 40):
        median_list.append(jaccardAndVolatilityAndConnectionViaGraphSimilarity(i, j))
median_prefAndJaccard = np.median(median_list)
print median_prefAndJaccard

In [ ]:
pred = []
count = 0
for index,sender,receiver,transaction in dftest.itertuples():
    combined_score = jaccardAndVolatilityScore(sender, receiver)
#     print combined_score
    if combined_score > 0.45:
        pred.append(1)
    else:
        pred.append(0)
#     print count
    count = count + 1

label = dftest['transaction']
fpr, tpr, thresholds = roc_curve(label, pred)
roc_auc = auc(fpr, tpr)
print "Area under the ROC curve : %f" % roc_auc

In [20]:
# Building latent structure
def classifyWithJaccart(S, R):
    ub = 1.5
    lb = 0.5
    penalty = 1
    bonus = 1
    outDegree_S = len(G[S])
    outDegree_R = len(G[R])
    inDegree_S = len(Gt[S])
    inDegree_R = len(Gt[R])
    if outDegree_S == 0:
        outDegree_S = ub + 10
    if outDegree_R == 0:
        outDegree_R = ub + 10
    ratio_S = float(inDegree_S) / float(outDegree_S)
    ratio_R = float(inDegree_R) / float(outDegree_R)
#     print ratio_S
#     print ratio_R
    if ratio_S < lb and lb < ratio_R < ub:
        score = bonus * jaccardSimilarity(S, R)
        print score
        return score
    elif lb < ratio_S < ub and not (ratio_R < lb):
        score = bonus * jaccardSimilarity(S, R)
        print score
        return score
    else:
        score = penalty * jaccardSimilarity(S, R)
        print score
        return score
        

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-20-65ce88dca079>, line 21)

In [ ]:
pred = []
count = 0
for index,sender,receiver,transaction in dftest.itertuples():
    combined_score = classifyWithJaccart(sender, receiver)
#     print combined_score
    if combined_score > 0.0001:
        pred.append(1)
    else:
        pred.append(0)
#     print count
    count = count + 1

label = dftest['transaction']
fpr, tpr, thresholds = roc_curve(label, pred)
roc_auc = auc(fpr, tpr)
print "Area under the ROC curve : %f" % roc_auc

In [7]:
# Page Rank algorithm
import networkx as nx

G_networkx = nx.DiGraph()
count_list = []
for (index, sender, receiver, count) in dftrain.itertuples():
    count_list.append(count)
    G_networkx.add_weighted_edges_from([(sender, receiver, count)])


In [8]:
G_networkx.size()

3348026

In [55]:
pr = nx.pagerank(G_networkx)

In [66]:
# find number of components
nx.number_strongly_connected_components(G_networkx)

9254

In [8]:
## Get Strongly Connected Components Data

import sys

number_nodes = len(G_networkx.nodes())

print(number_nodes)
sys.stdout.flush()

scc_gen = nx.strongly_connected_component_subgraphs(G_networkx)
print('computed components')
sys.stdout.flush()

#cout = 1
#scc_graphs = []
#for component in scc_gen:
#    print(cout, ":", len(nx.nodes(component))
#    sys.stdout.flush()
#    scc_graphs.append(component)
#    cout = cout + 1
#
#print('put components in list')
#sys.stdout.flush()
    
component_num = 0
v_scc_component = [-1]*number_nodes
v_scc_density = [-1]*number_nodes
for component in scc_gen:
    if (component_num % 1000 == 0):
        print(component_num)
    density = nx.density(component)
    for v in nx.nodes_iter(component):
        v_scc_component[v] = component_num
        v_scc_density[v] = density
    component_num = component_num + 1

444075
computed components
0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [14]:
cnt = 0
for v in range(number_nodes):
    if v_scc_component[v] == 1923:
        cnt = cnt + 1
print(cnt)
print(v_scc_density[100])
print(v_scc_component[100])

433676
1.7547270178143767e-05
1923


In [56]:
# now that we have calculated the page rank, we should investigate what this page rank represents
pr_median = np.median(pr.values())
print pr_median

6.30384417422e-07


In [57]:
pr_mean = np.mean(pr.values())
print pr_mean

2.25187186849e-06


In [60]:
pr_3rd_percentile = np.percentile(pr.values(), 100)
print pr_3rd_percentile

0.0808518536077


In [61]:
print max(pr_values)

0.0941045460319


In [43]:
pr_values = pr.values()
plt.hist(pr.values(), bins=np.arange(min(pr_values) - 0.0001, 0.01, 0.0001))
plt.show()

In [45]:
# use blind PageRank first
def pageRankLikelihood(S, R):
    prscore_R = pr[R]
    return prscore_R

In [52]:
pred = []
count = 0
for index,sender,receiver,transaction in dftest.itertuples():
    combined_score = pageRankLikelihood(sender, receiver)
#     print combined_score
    if combined_score > 0.0002:
        pred.append(1)
    else:
        pred.append(0)
#     print count
    count = count + 1

label = dftest['transaction']
fpr, tpr, thresholds = roc_curve(label, pred)
roc_auc = auc(fpr, tpr)
print "Area under the ROC curve : %f" % roc_auc

Area under the ROC curve : 0.468750


In [26]:
def cv_create():
    dftest.iloc[np.random.permutation(len(dftest))]
    l = int(len(dftest)/2)
    validate = dftest[:l]
    test = dftest[l+1:]

In [21]:
# Run a method with CV on a particular parameter
cv_create()

method_p = {}
for p in ParameterSpace:
    result = method(p, validate)
    method_p.add(p, result)
argmax = -1
best = -1
for k,v in method_p:
   if v > best:
        argmax = k
        best = v
# run test on argmax and print ROC curve

TypeError: cannot do slice indexing on <class 'pandas.indexes.range.RangeIndex'> with these indexers [5000.0] of <class 'float'>